In [2]:
import cv2 

img = cv2.imread("../input/sports-classification/images to predict/007.jpg")

img.shape

In [3]:
import matplotlib.pyplot as plt

plt.imshow(img)

## **Image Processing**

In [4]:
train_dir = "../input/sports-classification/train"
test_dir = "../input/sports-classification/test"
val_dir = "../input/sports-classification/valid"

In [5]:
import tensorflow as tf
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.,                                                                                                                   
                                                           rotation_range=0.2,
                                                           width_shift_range=0.2,
                                                           height_shift_range=0.2,
                                                           zoom_range = 0.2, 
                                                           horizontal_flip=True,
                                                           validation_split = 0.02)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)

val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)



In [6]:
train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           batch_size = 32
                                           )
test_data = test_gen.flow_from_directory(test_dir,
                                         target_size =(224,224),
                                         class_mode = "categorical",
                                         batch_size = 32
                                         )
val_data = val_gen.flow_from_directory(val_dir,
                                       target_size =(224,224),
                                       class_mode = "categorical",
                                       batch_size = 32
                                       )

In [7]:
labels = list(train_data.class_indices.keys())

len(labels)

In [8]:
# visualizing the train data

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for i in range(18):
  image,label = train_data.next()

  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(labels[tf.argmax(label[i])])
  plt.axis("off")

In [9]:
# building a CNN model

import tensorflow as tf
from tensorflow.keras import layers 


model = tf.keras.Sequential([
                             layers.Conv2D(filters= 64, kernel_size = 2, activation= "relu", input_shape = (224,224,3)),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Conv2D(filters= 64, kernel_size= 2, activation= "relu"),
                             layers.MaxPooling2D(pool_size = 2),

                             layers.Conv2D(filters= 64, kernel_size= 2, activation="relu"),
                             layers.MaxPooling2D(pool_size = 2),

                             layers.Flatten(),

                             layers.Dense(128, activation="relu"),
                             layers.Dense(73, activation="softmax")

])

In [10]:
model.summary()

In [11]:
# compiling the model

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [12]:
# fitting data to the model

model.fit(train_data,
          epochs = 10,
          steps_per_epoch = len(train_data),
          validation_data = val_data,
          validation_steps = len(val_data)
          )

In [13]:
model_evaluation = model.evaluate(test_data)

In [14]:
print(f"Model Accuracy: {model_evaluation[1] * 100 :0.2f} %")

In [15]:
# predicting and visualizing the test data

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for i in range(18):
  image,label = test_data.next()
  model_pred = model.predict(image)

  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(f"Prediction: {labels[tf.argmax(model_pred[i])]}\nOriginal: {labels[tf.argmax(label[i])]}")
  plt.subplots_adjust(top= 1.25)
  plt.axis("off")


In [16]:
# building a ResNet model
import tensorflow as tf

base_model = tf.keras.applications.ResNet50V2(include_top= False) 

base_model.trainable = False

In [17]:
# building a resnet Model

import tensorflow as tf
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3))

x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(73, activation="softmax")(x)

res_model = tf.keras.Model(inputs, outputs)


In [18]:
# compiling the model

res_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
    )

In [19]:
# fitting data to the model

res_model.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = val_data,
    validation_steps = len(val_data)
)

## **Tunning the model**

In [20]:
base_model.trainable = True

for layer in base_model.layers[:-3]:
  base_model.trainable = False

# compiling the model
res_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
    )
# fitting data

res_model.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = val_data,
    validation_steps = len(val_data)
)

In [21]:
res_model_evaluation = res_model.evaluate(test_data)

In [22]:
print(f"Resnet Model Accuracy: {res_model_evaluation[1] * 100 :0.2f} %")

In [24]:
# predicting and visualizing the test data

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for i in range(18):
  image,label = test_data.next()
  res_model_pred = res_model.predict(image)

  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(f"Prediction: {labels[tf.argmax(res_model_pred[i])]}\nOriginal: {labels[tf.argmax(label[i])]}")
  plt.subplots_adjust(top= 1.25)
  plt.axis("off")